Requirements : 
1. Need a system which can talk in a sarcasm, LLMs are generally trained on the public dataset, moslty availabel on internet. 
2. Need a chatbot which can use sarcasm therefore need a system using llms.
3. This above problem require to finetune a model on custom dataset, as the pre training is expensive and only left to the Big Tech.
4. Lets fine tune a model on our own dataset but this need a tighly packed modeles weights to trained on.

In [ ]:
!pip install datasets bitsandbytes trl huggingface-hub accelerate safetensors pandas matplotlib

In [2]:
import os
import torch
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTTrainer, SFTConfig

In [ ]:
support = torch.cuda.is_bf16_supported(including_emulation=False)
calculate_dtype = torch.bfloat16 if support else torch.float32
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.float32, #calculate_dtype can be bf16 or float32- use bf16 if supported
    bnb_4bit_use_double_quant= True
    )
repo = "microsoft/Phi-3-mini-4k-instruct"
model = AutoModelForCausalLM.from_pretrained(repo, quantization_config= bnb_config, device_map= "cuda:0")


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
# import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# torch.random.manual_seed(0)
# model = AutoModelForCausalLM.from_pretrained(
#     "microsoft/Phi-3-mini-4k-instruct",
#     device_map="cuda",
#     torch_dtype="auto",
#     trust_remote_code=True,
# )

# tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

# messages = [
#     {"role": "system", "content": "You are a helpful AI assistant."},
#     {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"},
#     {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."},
#     {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"},
# ]

# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
# )

# generation_args = {
#     "max_new_tokens": 500,
#     "return_full_text": False,
#     "temperature": 0.0,
#     "do_sample": False,
# }

# output = pipe(messages, **generation_args)
# print(output[0]['generated_text'])


In [4]:
print(model.get_memory_footprint()/1024/1024)

2104.1310424804688


In [5]:
model

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear4bit(in_features=3072, out_features=9216, bias=False)
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear4bit(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (norm): Phi3RMSNorm((3072,), eps=1e-05)
    (rotary_emb): Phi3RotaryEmbedding()
  )
  (lm_head): Linear(in_features=3072, out_

In [6]:
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r = 8, #. rank of LoRA - [4-16]
    bias = "none", # ["all", "lora_only"] - for train bias term
    lora_alpha = 16, # scalling factor
    lora_dropout = 0.05, # prevent overfit- used for regularisation
    target_modules = ["query_key_value", "o_proj", "qkv_proj", "gate_up_proj", "down_proj"],
    task_type = "CAUSAL_LM"

)

model = get_peft_model(model, config)
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Phi3ForCausalLM(
      (model): Phi3Model(
        (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
        (layers): ModuleList(
          (0-31): 32 x Phi3DecoderLayer(
            (self_attn): Phi3Attention(
              (o_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (

In [7]:
print(model.get_memory_footprint()/1024/1024)

2528.2619018554688


In [8]:
print(model.get_base_model)

<bound method PeftModel.get_base_model of PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Phi3ForCausalLM(
      (model): Phi3Model(
        (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
        (layers): ModuleList(
          (0-31): 32 x Phi3DecoderLayer(
            (self_attn): Phi3Attention(
              (o_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): Mo

In [9]:
print(model.get_memory_footprint()/1e6)

2651.074752


In [10]:
trainable_params, total_params = model.get_nb_trainable_parameters()
percentage = (trainable_params / total_params) * 100

print(f"Trainable Parameters: {trainable_params:,}")
print(f"Total Parameters: {total_params:,}")
print(f"Percentage Trainable: {percentage:.2f}%")

Trainable Parameters: 12,582,912
Total Parameters: 3,833,662,464
Percentage Trainable: 0.33%


In [ ]:
dataset = load_dataset("sweatSmile/sarcastic-dataset", split="train")
dataset

In [12]:
dataset[0]

{'sentence': 'The birch canoe slid on the smooth planks.',
 'translation': 'Oh, look at that—a birch canoe gliding effortlessly across those impeccably smooth planks. How incredibly thrilling.',
 'translation_extra': "Oh, wow, a birch canoe slid on smooth planks? Who would have thought that would happen? Next, you'll tell me water's wet and the sky is blue!"}

In [13]:
dataset = dataset.rename_column("sentence", "prompt")
dataset = dataset.rename_column("translation_extra", "completion")
dataset = dataset.remove_columns(["translation"])
dataset

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 720
})

In [14]:
dataset[0]

{'prompt': 'The birch canoe slid on the smooth planks.',
 'completion': "Oh, wow, a birch canoe slid on smooth planks? Who would have thought that would happen? Next, you'll tell me water's wet and the sky is blue!"}

In [16]:
messages = [
    {"role": "user", "content": dataset[0]['prompt']},
    {"role": "assistant", "content": dataset[0]['completion']}
]
messages

[{'role': 'user', 'content': 'The birch canoe slid on the smooth planks.'},
 {'role': 'assistant',
  'content': "Oh, wow, a birch canoe slid on smooth planks? Who would have thought that would happen? Next, you'll tell me water's wet and the sky is blue!"}]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(repo)
tokenizer.chat_template

In [18]:
print(tokenizer.apply_chat_template(messages, tokenize=False))

<|user|>
The birch canoe slid on the smooth planks.<|end|>
<|assistant|>
Oh, wow, a birch canoe slid on smooth planks? Who would have thought that would happen? Next, you'll tell me water's wet and the sky is blue!<|end|>
<|endoftext|>


In [ ]:
sft_config = SFTConfig(
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={'use_reentrant': True},  # <-- fixed here (False)
    gradient_accumulation_steps=1,
    per_device_train_batch_size=16,
    auto_find_batch_size=True,
    max_seq_length=64,
    packing=True,
    num_train_epochs=10,
    learning_rate=3e-4,
    optim='paged_adamw_8bit',
    logging_steps=10,
    logging_dir="/content/drive/MyDrive/phi3-mini-sarcasm/logs",
    output_dir="/content/drive/MyDrive/phi3-mini-sarcasm/adapter",
    report_to="none"
)


In [30]:
trainer=SFTTrainer(
    model=model,
    train_dataset=dataset,
    processing_class=tokenizer,
    args=sft_config
)

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:412: UserWarning: Padding-free training is enabled, but the attention implementation is not set to 'flash_attention_2'. Padding-free training flattens batches into a single sequence, and 'flash_attention_2' is the only known attention mechanism that reliably supports this. Using other implementations may lead to unexpected behavior. To ensure compatibility, set `attn_implementation='flash_attention_2'` in the model configuration, or verify that your attention mechanism can handle flattened sequences.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:458: UserWarning: You are using packing, but the attention implementation is not set to 'flash_attention_2'. Packing flattens batches into a single sequence, and 'flash_attention_2' is the only known attention mechanism that reliably supports this. Using other implementations may lead to cross-contamination between batches. To avoid this, ei

In [31]:
dl=trainer.get_train_dataloader()
batch = next(iter(dl))

In [32]:
batch['input_ids'][0], batch['labels'][0]

(tensor([  319,  2686,   479,   674, 26755,   278,  4549, 26406, 29889,  9048,
         29892, 27592,  3814, 29991,  7311, 29892,   366,  1073, 29892,   263,
          2686,   479,   756,   334,   484,   369, 29930,  1063,  2998,   304,
         26755,   263,  4549, 26406,  1434, 29889,  1605, 11850,  5962,  1030,
          5086,  6433,  1244, 29991, 32000,   450,   767, 26758,   263,  1238,
          1624,   297,   670,  7091,  3056, 29889,  9048, 29892,  1363,   310,
          3236, 29892,   297,   263,  3186, 10423,   411,  5962,  1030,  5086,
         13460, 19995, 29892,   278,   282,  2559,  6436,   310,  3114,   338,
          9436,   263,  2323,  1238,  1624,   297,   263,  7091,  3056, 29889,
          1605, 11850, 19479,   653, 29991, 32000,   450, 25008,  1898,   304,
         14074,   670,  1206, 29889,  9048, 29892,   310,  3236, 29892,  1363,
          1432, 25008, 12561, 29879,   310,   278,  2462,   896,   508, 22314,
           368,  2317,   297,  8973,   322,  1827, 2

In [33]:
trainer.train()

Step,Training Loss
10,2.339000
20,1.896000
30,1.759700
40,1.802800
50,1.688300
60,1.539300
70,1.510300
80,1.522200
90,1.512800
100,1.214100


TrainOutput(global_step=450, training_loss=0.6869243285391066, metrics={'train_runtime': 608.5035, 'train_samples_per_second': 11.717, 'train_steps_per_second': 0.74, 'total_flos': 7742318198353920.0, 'train_loss': 0.6869243285391066})

In [34]:
def gen_prompt(tokenizer, sentence):
  converted_sample = [{"role": "user", "content": sentence}]
  prompt = tokenizer.apply_chat_template(converted_sample, tokenize=False, add_generation_prompt=True)
  return prompt


In [35]:
sentence = 'The birch canoe slid on the smooth planks.'
prompt = gen_prompt(tokenizer, sentence)
prompt

'<|user|>\nThe birch canoe slid on the smooth planks.<|end|>\n<|assistant|>\n'

In [37]:
def generate(model, tokenizer, prompt, max_new_tokens=64, skip_special_tokens=True):
  tokenized_input = tokenizer(prompt,add_special_tokens=False, return_tensors="pt").to(model.device)
  model.eval()
  generation_output = model.generate(**tokenized_input, max_new_tokens=max_new_tokens, eos_token_id=tokenizer.eos_token_id)
  output = tokenizer.batch_decode(generation_output, skip_special_tokens=skip_special_tokens)[0]
  return output

In [38]:
print(generate(model, tokenizer, prompt))

The birch canoe slid on the smooth planks. Ah, yes, the birch canoe slid on the smooth planks—because who wouldn't expect wood to float and glide like it's auditioning for a soap opera?


In [39]:
sentence = 'The Deployment has crashed again on friday night.'
prompt = gen_prompt(tokenizer, sentence)
prompt

'<|user|>\nThe Deployment has crashed again on friday night.<|end|>\n<|assistant|>\n'

In [40]:
def generate(model, tokenizer, prompt, max_new_tokens=64, skip_special_tokens=True):
  tokenized_input = tokenizer(prompt,add_special_tokens=False, return_tensors="pt").to(model.device)
  model.eval()
  generation_output = model.generate(**tokenized_input, max_new_tokens=max_new_tokens, eos_token_id=tokenizer.eos_token_id)
  output = tokenizer.batch_decode(generation_output, skip_special_tokens=skip_special_tokens)[0]
  return output

In [41]:
print(generate(model, tokenizer, prompt))

The Deployment has crashed again on friday night. Oh, fantastic! Because, what else could I possibly predict for you today but the blissful spectacle of the deployment crashing yet again on a Friday night? Truly, my day is complete!


In [42]:
trainer.save_model('ak-phi3-mini-sarcasm-adapter')